# Stencils

### Timing

In [1]:
import timeit
import numpy as np
import cupy as cp

In [2]:
tic = timeit.default_timer()
toc = timeit.default_timer()
elapsed_time_work = toc - tic
print(elapsed_time_work)

In [3]:
#sinchronisation of CPU and GPU
def get_time(sync=True):
    if sync:
        cp.cuda.Device().synchronize()
    return timeit.default_timer()

### pointwise stencil:
notebook day 5: 01-GT4Py-sumdiff.ipynb
-->comparison between Numpy, CuPy, GT4Py

In [4]:
import numpy as np

shape = (512, 512, 128)

def f_numpy(a, b, c, d):
    d[...] = a + b - c
    
a = np.random.rand(*shape)
b = np.random.rand(*shape)
c = np.random.rand(*shape)
d = np.empty_like(a)

%timeit f_numpy(a, b, c, d)

192 ms ± 693 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
##simpler pointwise stencil in GT4Py
def to_one_defs(a: gtscript.Field[float]):
    # TODO: write the stencil definition
    from __gtscript__ import PARALLEL, computation, interval
    
    with computation(PARALLEL), interval(...):
        a=1

In [10]:
#in numpy
import numpy as np

shape = (512, 512, 128)

def f_numpy(a):
    a[...] = 1
    
a = np.random.rand(*shape)

%timeit f_numpy(a)
#a.view()

25 ms ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 1D-stencil: Fibonacci
notebook day 5: 03-GT4Py-concepts.ipynb

In [ ]:
#GT4Py
def fibonacci_defs(inout_field: gtscript.Field[float]):
    from __gtscript__ import BACKWARD, FORWARD, PARALLEL, computation, interval
    
    with computation(FORWARD):
        with interval(0, 2):
            inout_field = 1.0
        with interval(2, None):
            inout_field = inout_field[0, 0, -1] + inout_field[0, 0, -2]

In [ ]:
fibonacci =  gtscript.stencil(definition=fibonacci_defs, backend="gtmc")

In [26]:
#numpy:
#in numpy
import numpy as np

shape = (512,512 ,128)

def fibonacci_numpy(a):
    a[:,:,0] = 1.0
    a[:,:,1] = 1.0
    for i in np.arange(2,a.shape[2]):
        a[:,:,i] =  a[:,:,i-1] +  a[:,:,i-2]
    
a = np.zeros(shape)

%timeit fibonacci_numpy(a)
#a.view()

375 µs ± 83.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 2D-stencil: Laplace
notebook day 5: 02-GT4Py-laplacian-->-->comparison between Numpy, CuPy (C-style), CuPy (F-style), GT4Py

In [5]:
import numpy as np

shape = (512, 512, 128)

def lap_numpy(phi, lap):
    lap[1:-1, 1:-1] = (
        - 4. * phi[1:-1, 1:-1]
        +      phi[ :-2, 1:-1]
        +      phi[  2:, 1:-1]
        +      phi[1:-1,  :-2]
        +      phi[1:-1,   2:]
    )
    
phi = np.random.rand(*shape)
lap = np.empty_like(phi)

%timeit lap_numpy(phi, lap)

300 ms ± 413 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


from day 1: stencil2d-counter.F90.-->2D stencil

In [ ]:
subroutine laplacian( field, lap, num_halo, extend, increase_counters )
        implicit none
            
        ! argument
        real (kind=wp), intent(in) :: field(:, :, :)
        real (kind=wp), intent(inout) :: lap(:, :, :)
        integer, intent(in) :: num_halo, extend
        logical, intent(in) :: increase_counters
        
        ! local
        integer :: i, j, k
            
        do k = 1, nz
        do j = 1 + num_halo - extend, ny + num_halo + extend
        do i = 1 + num_halo - extend, nx + num_halo + extend
            lap(i, j, k) = -4._wp * field(i, j, k)      &
                + field(i - 1, j, k) + field(i + 1, j, k)  &
                + field(i, j - 1, k) + field(i, j + 1, k)
            ! TODO
        end do
        end do
        end do

    end subroutine laplacian


In [ ]:
size =10
for backend in zip([np, cp], ["numpy", "cupy"]):

    #simple stencil
    phi = xp.random.rand(size)
    nan = xp.array( [xp.nan] )
    phi= xp.concatenate( (nan, phi, nan) )
    phi_new = xp.empty_like(phi)
    d_x = 1

    #i=np.arange(...) would also be vectorized
    tic = get_time()
    phi_new=(phi[:-2]+2*phi[1:-1] +phi[2:])/d_x
    elapsed_time_simple = get_time() - tic
    print(phi_new)

    #upstream scheme
    u=xp.random.rand(size)-xp.repeat(0.5,size)
    phi = xp.random.rand(size)
    nan = xp.array( [xp.nan] )
    phi= xp.concatenate( (nan, phi, nan) )
    phi_new = xp.empty_like(phi)
    d_x = 1
    phi_new=xp.where(u>=0, u*(phi[1:-1]-phi[:-2])/d_x, u*(phi[0:-2]-phi[1:-1])/d_x)#[u<0], [u>=0]*
    print(phi)
    print(u)
    print(phi_new)

    print(
        backend, elapsed_time_simple
    ) 